# Grounding in Logic Tensor Networks (LTN)

LTN interprets symbols which are grounded on real-valued features, we use the term grounding G. G associates a tensor of real numbers to any term of the language, and a real number in the interval to any formula f.

The language consists of a non-logical part (the signature) and logical connectives and quantifiers.

In [1]:
%pip install ltn

INFO: pip is looking at multiple versions of ltn to determine which version is compatible with other requirements. This could take a while.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tensorflow-2.20.0-cp313-cp313-macosx_12_0_arm64.whl.metadata (4.5 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-1-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached setuptools-80.10.2-py3-none-any.whl.metadata (6.6 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached wrapt-2.0.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.0 kB)
  Using cached grpcio-1.76.0-cp313-cp313-macosx_11_0_un

## Grounding in Logical Tensor Networks

In [2]:
import ltn
import tensorflow as tf
import numpy as np

In [ ]:
# Constants

c1 = ltn.Constant([2.1, 3] , trainable=False)
c2 = ltn.Constant([[4.2,3,2.5],[4,-1.3,1.8]], trainable=False)

#Note that a constant can be set as learnable by using the keyword
#argument trainable=True. This is useful to learn embeddings for
#some individuals. The features of the tensor will be considered as
#trainable parameters

In [8]:
c3 = ltn.Constant([0.,0.] ,trainable=True)
#You can access the TensorFlow value of a LTN constant or any LTN expression x by querying x.tensor.

In [10]:
print(c1)
print(c1.tensor)
print(c2)
print(c2.tensor)
print(c3)
print(c3.tensor)

ltn.Constant(tensor=[2.1 3. ], trainable=False, free_vars=[])
tf.Tensor([2.1 3. ], shape=(2,), dtype=float32)
ltn.Constant(tensor=[[ 4.2  3.   2.5]
 [ 4.  -1.3  1.8]], trainable=False, free_vars=[])
tf.Tensor(
[[ 4.2  3.   2.5]
 [ 4.  -1.3  1.8]], shape=(2, 3), dtype=float32)
ltn.Constant(tensor=<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>, trainable=True, free_vars=[])
<tf.Variable 'Variable:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>


In [11]:
#predicates

mu = tf.constant([2.,3.])
P1 = ltn.Predicate.Lambda(lambda x : tf.exp(-tf.norm(x-mu,axis=1)))

class ModelP2(tf.keras.Model):
    #"https://www.tensorflow.org/api_docs/python/tf/keras/Model"
    def __init__(self):
        super(ModelP2, self).__init__()
        self.dense1 = tf.keras.layers.Dense(5, activation=tf.nn.elu)
        self.dense2 = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid) # returns one value in [0,1]
    def call(self, x):
        x = self.dense1(x)
        return self.dense2(x)

modelP2 = ModelP2()
P2 = ltn.Predicate(modelP2)


In [ ]:
#One can easily query predicates using LTN constants and LTN variables
c1 = ltn.Constant([2.1,3],trainable=False)
c2 = ltn.Constant([4.5,0.8],trainable=False)

print(P1(c1))

ltn.Formula(tensor=0.9048374891281128, free_vars=[])


In [14]:
class ModelP3(tf.keras.Model):
    def __init__(self):
        super(ModelP3, self).__init__()
        self.dense1 = tf.keras.layers.Dense(5, activation=tf.nn.elu)
        self.dense2 = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid) # returns one value in [0,1]
        
    def call(self, inputs):
        x1, x2 = inputs[0], inputs[1] # multiple arguments are passed as a list
        x = tf.concat([x1,x2],axis=1) # axis=0 is the batch axis
        x = self.dense1(x)
        return self.dense2(x)
    
P3 = ltn.Predicate(ModelP3())
print(P3([c1,c2])) # multiple arguments are passed as a list

ltn.Formula(tensor=0.9872696995735168, free_vars=[])


In [15]:

# Declaring a trainable 0-ary predicate with initial truth value 0.3
A = ltn.Proposition(0.3, trainable=False)
print(A)

ltn.Proposition(tensor=0.30000001192092896, trainable=False, free_vars=[])


In [16]:
# Functions

"""
the default constructor ltn.Function(model) takes in argument a tf.keras.Model instance; it can be used to ground any custom function (succession of operations, Deep Neural Network, ...),
the lambda constructor ltn.Function.Lambda(function) takes in argument a lambda function; it is appropriate for small mathematical operations with no weight tracking (non-trainable function).
"""

f1 = ltn.Function.Lambda(lambda args: args[0]-args[1])

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(4, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(5)
    def call(self, x):
        x = self.dense1(x)
        return self.dense2(x)

model_f2  = MyModel()
f2 = ltn.Function(model_f2)




In [17]:

c1 = ltn.Constant([2.1,3], trainable=False)
c2 = ltn.Constant([4.5,0.8], trainable=False)
print(f1([c1,c2])) # multiple arguments are passed as a list
print(f2(c1))

ltn.Term(tensor=[-2.4  2.2], free_vars=[])
ltn.Term(tensor=[-1.1321932 -0.9576021  0.1673333 -1.9088415 -0.2597782], free_vars=[])


In [ ]:
#variables

"""
TN variables are sequences of individuals/constants from a domain. Variables are useful to write quantified statements
"""

#The following defines two variables and x and y
# with respectively 10 and 5 individuals, sampled from normal distributions 

x = ltn.Variable('x', np.random.normal(0.,1.,(10,2)))
y = ltn.Variable('y',np.random.normal(0.,4.,(5,2)))

print(x)


ltn.Variable(label=x, tensor=[[-0.6622027   2.3706868 ]
 [-0.7119895   1.0917885 ]
 [ 1.7085938   0.868343  ]
 [-0.23839763  2.2870016 ]
 [-0.81183326 -1.2873626 ]
 [-1.0229675  -0.88876134]
 [ 1.0546168   1.5523195 ]
 [-0.6659683  -0.02354109]
 [ 0.1472501  -0.4015604 ]
 [-0.40952152 -1.0498368 ]], free_vars=['x'])


In [19]:
# Notice that the outcome is a 2 dimensional tensor where each cell
# represents the satisfiability of P3 with each individual in x and in y.
res1 = P3([x,y])
print(res1) 
print(res1.take('x',2).take('y',0)) # gives the result calculated with the 3rd individual in x and the 1st individual in y

ltn.Formula(tensor=[[0.9445251  0.99786943 0.6884894  0.34428936 0.38078645]
 [0.91775936 0.9968835  0.6011971  0.26200792 0.29250276]
 [0.90213776 0.99651223 0.5480611  0.17460628 0.19977736]
 [0.9432688  0.9978344  0.68200684 0.32575727 0.3619223 ]
 [0.7989366  0.992503   0.419      0.13307758 0.15446018]
 [0.83140767 0.99375004 0.46359465 0.15700844 0.181491  ]
 [0.92615503 0.9973126  0.6162216  0.23719613 0.26619172]
 [0.87712383 0.9954754  0.51440614 0.19392799 0.21958849]
 [0.8514917  0.99459153 0.45052028 0.15094696 0.1726312 ]
 [0.812923   0.99308664 0.41767448 0.13424884 0.1548102 ]], free_vars=['x', 'y'])
ltn.Formula(tensor=0.9021377563476562, free_vars=[])


In [20]:
# This is also valid with the outputs of `ltn.Function`
res2 = f1([x,y])
print(res2.tensor.shape)
print(res2.free_vars)
print(res2.take('x',2).take('y',0)) # gives the result calculated with the 3rd individual in x and the 1st individual in y

(10, 5, 2)
['x', 'y']
ltn.Term(tensor=[-1.2298062  0.752656 ], free_vars=[])


In [22]:

res3 = P3([c1,y])
print(res3)

ltn.Formula(tensor=[0.9539946  0.99829996 0.72398144 0.30053917 0.3466746 ], free_vars=['y'])


In [23]:
#Variables made of trainable constants

c1 = ltn.Constant([2.1,3], trainable=True)
c2 = ltn.Constant([4.5,0.8], trainable=True)

with tf.GradientTape() as tape:
    # the assignation must be done within a tf.GradientTape.
    # Tensorflow will keep track of the gradients between c1/c2 and x.
    x = ltn.Variable.from_constants("x", [c1,c2], tape=tape)
    res = P2(x)
tape.gradient(res.tensor,c1.tensor).numpy() # the tape keeps track of gradients between P2(x), x and c1


array([-0.0023127, -0.0438941], dtype=float32)